In [1]:
import pandas as pd
import requests
import os
import dotenv   
from bs4 import BeautifulSoup
# turn off warnings
import warnings
warnings.filterwarnings('ignore')

### Get an api key from : https://apilayer.com/marketplace/google_search-api 

### Then set it to a variable named 'google' in your .env file 

In [2]:
# import .env file
dotenv.load_dotenv()

True

In [3]:
# list of all 50 US states
united_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware',
                    'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
                    'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                    'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
                    'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                    'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
                    'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

### Below is how we query the api and get the results split into the first 10 results and the next 90 results

In [4]:
# for state in united_states:
#   q = f'{state}%20storm%20article'
#   url = f"https://api.apilayer.com/google_search?q={q}"

#   payload = {}
#   headers= {
#   "apikey": os.getenv('google'),
#   }

#   response = requests.request("GET", url, headers=headers, data = payload)

#   status_code = response.status_code
#   result = response.text
# requests folder holds the json files for the first page of results for each state
#   with open(f'requests/{state}.json', 'w') as f:
#         f.write(response.text)
#   print(f'{state} status code: {status_code}')

In [5]:
# for state in united_states:
#   for i in range(10):

#       q = f'{state}%20storm%20article'
#       url = f"https://api.apilayer.com/google_search?q={q}&start={11+i}"

#       payload = {}
#       headers= {
#       "apikey": os.getenv('google'),
#       }

#       response = requests.request("GET", url, headers=headers, data = payload)

#       status_code = response.status_code
#       result = response.text
#       name = f'{state}_{i}'
# more_requests folder holds the json files for pages 2-10 of results for each state
#       with open(f'more_requests/{name}.json', 'w') as f:
#             f.write(response.text)
#       print(f'{state}_{i} status code: {status_code}')

### below gets the urls found in the requests and stores them in a list organized by State

In [6]:
import json
# urls
urls = {}

# step through the json files and get the urls for the articles
for file in os.listdir('./requests/'):
    with open(f'./requests/{file}') as f:
        urls[file[:-5]] = []
        data = json.load(f)
        for item in data['organic']:
            url = item['link']
            urls[file[:-5]].append(url)
urls

            

{'West Virginia': ['https://www.wowktv.com/top-stories/rounds-of-storms-for-west-virginia-kentucky-and-ohio/',
  'https://wvmetronews.com/2023/03/24/storm-headed-into-west-virginia-packs-a-punch/',
  'https://globalnews.ca/tag/west-virginia-storm/',
  'https://www.usatoday.com/story/news/nation/2023/02/17/west-virginia-weather-flooding-state-emergency/11279958002/',
  'https://www.weather.gov/rlx/snow',
  'https://wvpublic.org/strong-winds-storms-expected-across-the-state-saturday/',
  'https://www.wvgazettemail.com/news/wvs-eastern-mountains-brace-for-rare-may-snowstorm/article_0673c102-e6ef-5eb4-9454-b3882569a86e.html',
  'https://www.wboy.com/weather/say-it-aint-snow-west-virginia-mountains-to-see-few-inches-of-snow-in-early-may/',
  'https://stormhighway.com/wvstorms.php'],
 'Texas': ['https://www.texastribune.org/series/winter-storm-power-outage/',
  'https://www.nytimes.com/2023/01/31/us/ice-storm-winter-weather-texas.html',
  'https://www.cbsnews.com/news/texas-storms-heavy-rain

In [7]:
# add more reqeusts to the urls dictionary
for file in os.listdir('./more_requests/'):
    with open(f'./more_requests/{file}') as f:
        # urls[file[:-5]] = []
        data = json.load(f)
        for item in data['organic']:
            url = item['link']
            urls[file[:-7]].append(url)

In [8]:
# print the total number of urls
total = 0
for key in urls.keys():
    total += len(urls[key])
print(total)

5199


### below the urls are iterated through and the webpages are downloaded and stored in a folder named 'articles'

In [10]:
# download the articles from the urls in parallel
articles = os.listdir('./articles')

def download_article(state, i, url):
    print(f'{state}_{i}')
    if f'{state}_{i}.html' in articles:
        return
    try:
        # stop requests from hanging
        r = requests.get(url, timeout=5)
        
        
        with open(f'./articles/{state}_{i}.html', 'w') as f:
            f.write(r.text)
    except:
        print(f'error with {state}_{i}')

from joblib import Parallel, delayed
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(num_cores)

downloads = Parallel(n_jobs=num_cores)(delayed(download_article)(state, i, url) for state in urls.keys() for i, url in enumerate(urls[state]))



12
West Virginia_0
West Virginia_2
West Virginia_1
West Virginia_6
West Virginia_7
West Virginia_8
West Virginia_9
West Virginia_10
West Virginia_11
West Virginia_3
West Virginia_12
West Virginia_13
West Virginia_14
West Virginia_15
West Virginia_16
West Virginia_18
West Virginia_19
West Virginia_20
West Virginia_21
West Virginia_23
West Virginia_24
West Virginia_25
West Virginia_28
West Virginia_29
West Virginia_4
West Virginia_30
West Virginia_31
West Virginia_33
West Virginia_34
West Virginia_5
West Virginia_35
West Virginia_36
West Virginia_37
West Virginia_38
West Virginia_39
West Virginia_40
West Virginia_41
West Virginia_42
West Virginia_43
West Virginia_44
West Virginia_46
West Virginia_47
West Virginia_48
West Virginia_49
West Virginia_50
West Virginia_51
West Virginia_52
West Virginia_53
West Virginia_54
West Virginia_55
West Virginia_56
West Virginia_57
West Virginia_17
West Virginia_58
West Virginia_59
West Virginia_60
West Virginia_61
West Virginia_62
West Virginia_63
West

### below the articles are read and the text is extracted and stored in a dataframe 

In [16]:
# organize the articles into a dataframe
articles = os.listdir('./articles')

# create a dataframe
columns = ['title', 'author', 'publication', 'body_text' , 'url', 'state']
df = pd.DataFrame(columns=columns)

# step through the articles and add them to the dataframe

for article in articles:
    with open(f'./articles/{article}') as f:
        soup = BeautifulSoup(f.read(), 'html.parser')
        title = soup.find('title')
        if title:
            title = title.text
        author = soup.find('meta', {'name': 'author'})
        if author:
            author = author['content']
        publication = soup.find('meta', {'name': 'publication'})
        if publication:
            publication = publication['content']
        body_text = soup.find('body')
        if body_text:
            body_text = body_text.text
        url = soup.find('meta', {'property': 'og:url'})
        if url:
            url = url['content']
        state = article.split('_')[0]
        df = df.append({'title': title, 'author': author, 'publication': publication, 'body_text': body_text, 'url': url, 'state': state}, ignore_index=True)

df

/var/folders/gv/857l914131304z8vlfgswdg80000gn/T/ipykernel_1751/3430119940.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': title, 'author': author, 'publication': publication, 'body_text': body_text, 'url': url, 'state': state}, ignore_index=True)
/var/folders/gv/857l914131304z8vlfgswdg80000gn/T/ipykernel_1751/3430119940.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': title, 'author': author, 'publication': publication, 'body_text': body_text, 'url': url, 'state': state}, ignore_index=True)
/var/folders/gv/857l914131304z8vlfgswdg80000gn/T/ipykernel_1751/3430119940.py:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'title': title, 'author': author,

,title,author,publication,body_text,url,state
0,Access to this page has been denied,None,None,\n\n\n,None,Michigan
1,Tornadoes severe weather updates: 26 dead in M...,None,None,Start the day smarter ☀️ How often do women gi...,https://www.usatoday.com/story/news/nation/202...,Mississippi
2,Fatality confirmed in Vermont flooding amid ex...,ABC News,None,\nABC NewsVideoLiveShowsElectionsInterest Succ...,https://abcnews.go.com/US/tornadoes-midwest-fl...,Illinois
3,Terrifying moment monster tornado rips apart A...,Aneeta Bhole,None,\n\n\n \n\n\n\nHome\n\n\n\n\nShowbiz\n\n\n\n\n...,https://www.dailymail.co.uk/news/article-11926...,Arkansas
4,Maryland weather: Storms cease Tuesday night a...,None,None,Close this dialog This website stores data su...,https://www.baltimoresun.com/weather/bs-md-thu...,Maryland
...,...,...,...,...,...,...
5007,Winter Storm Approaching Idaho; Expert Boise S...,None,None,\n\n\n\nSkip to main contentSkip to site foote...,https://liteonline.com/major-winter-storm-appr...,Idaho
5008,New Mexico — Weather,None,None,\n\n\n\n\n\n\n\nBUCKET LISTS\n\n\nTRIP FINDER\...,https://www.iexplore.com/articles/travel-guide...,New Mexico
5009,Colorado Climate | Articles | Colorado Encyclo...,None,None,\n\n\n\n\nSkip to main content\n\nPlease enabl...,None,Colorado
5010,Snow maps: Massachusetts forecast for March 3-...,Dialynn Dwyer,None,\n \n\n\n \n\n\t\t\t\tSkip to Main Content\t\t...,https://www.boston.com/weather/weather/2023/03...,Massachusetts


In [ ]:
# Pickle file for use on part 2
df = pd.to_pickle('articles.pkl')
# df

# Phase 1 complete